In [3]:
import pandas as pd
import numpy as np
df=pd.read_csv('dataframe_train.csv')
df.head()

In [4]:
df.info()

In [5]:
print(df.columns)

In [32]:
print(len(df['source_type'].unique()))

In [8]:
df.describe()

In [13]:
print(df['weather_grade'].unique())

In [122]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 加载训练数据
df_train = pd.read_csv("dataframe_train.csv")

# 选择初始特征
selected_features = ['courier_id', 'wave_index', 'courier_wave_start_lng',
       'courier_wave_start_lat', 'date', 'group', 'level',
       'speed', 'max_load', 'weather_grade', 'aoi_id', 'shop_id',
       'source_type', 'source_tracking_id', 'source_lng', 'source_lat',
       'target_lng', 'target_lat', 'grid_distance', 
       'urgency']

# 提取特征和目标变量
X = df_train[selected_features]
y = df_train['action_type']  # 目标变量（DELIVERY/PICKUP）

In [123]:
mapping = {
        'DELIVERY': 1,
        'PICKUP': 0,
        'ASSIGN':2
}
y = df_train['action_type'].map(mapping)

In [124]:
    
def pre(X):
    X['date'] = pd.to_datetime(X['date'], format='%Y%m%d')
    
    # 提取日期特征
    X['day_of_week'] = X['date'].dt.dayofweek  # 星期几（0=周一，6=周日）
    X['is_weekend'] = X['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)  # 是否周末
    
    # 删除原始日期列
    X = X.drop(columns=['date'])
    # 计算每个 aoi_id 的出现频率
    aoi_freq = X['aoi_id'].value_counts(normalize=True).to_dict()
    X['aoi_freq'] = X['aoi_id'].map(aoi_freq)
    
    # 计算每个 shop_id 的出现频率
    shop_freq = X['shop_id'].value_counts(normalize=True).to_dict()
    X['shop_freq'] = X['shop_id'].map(shop_freq)
    
    # 删除原始高基数列
    X = X.drop(columns=['aoi_id', 'shop_id'])
    X = X.drop(columns=['courier_id'])
    
    # 定义天气等级的顺序（根据业务逻辑调整）
    weather_order = ['Normal Weather' ,'Slightly Bad Weather','Bad Weather' ,'Very Bad Weather']
    X['weather_grade'] = X['weather_grade'].map({k: v for v, k in enumerate(weather_order)})
    
    mapping = {
        'DELIVERY': 1,
        'PICKUP': 0,
        'ASSIGN':2
    }
    
    X['source_type']=X['source_type'].map(mapping)
    return X
X=pre(X)

In [107]:
# 合并特征和目标变量后下采样
df_sampled = pd.concat([X, y], axis=1).sample(n=40000, random_state=42)

# 重新分离特征和目标
X_sampled = df_sampled.drop(columns=['action_type'])
y_sampled = df_sampled['action_type']

In [108]:
print(y_sampled)

In [100]:
mapping = {
    'DELIVERY': 1,
    'PICKUP': 0
}
y_sampled = y_sampled.map(mapping)

In [ ]:
x=X_sampled[['wave_index', 'courier_wave_start_lng', 'courier_wave_start_lat',
        'level', 'speed', 'max_load', 'weather_grade', 'source_type',
        'source_lng', 'source_lat', 'target_lng',
       'target_lat', 'grid_distance',  'urgency',
       'day_of_week', ]]

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# 标准化数值特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 训练逻辑回归模型
model = LogisticRegression(max_iter=1000)
f1_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='f1')

print("基准模型 F1 分数均值:", f1_scores.mean())
print("各折 F1 分数:", f1_scores)

In [86]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt

# 假设 x 是特征数据，y_sampled 是目标变量
# 标准化数值特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sampled)

# 初始化随机森林分类器
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# 使用交叉验证评估模型性能
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = cross_val_score(rf_model, X_scaled, y_sampled, cv=cv, scoring='f1')

print("随机森林模型 F1 分数均值:", f1_scores.mean())
print("各折 F1 分数:", f1_scores)

# 再次训练模型以获取特征重要性
rf_model.fit(X_scaled, y_sampled)



In [110]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt

# 假设 x 是特征数据（DataFrame），y_sampled 是目标变量
# 标准化数值特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sampled)

# 划分训练集和测试集（80% 训练集，20% 测试集）
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_sampled, test_size=0.2, random_state=42)

# 初始化随机森林分类器
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# 使用交叉验证评估模型性能（仅在训练集上进行交叉验证）
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='f1')

print("随机森林模型 F1 分数均值:", f1_scores.mean())
print("各折 F1 分数:", f1_scores)

# 在训练集上训练模型
rf_model.fit(X_train, y_train)

# 在测试集上评估模型
y_pred = rf_model.predict(X_test)
print("\n测试集 F1 分数:", f1_score(y_test, y_pred))
print("分类报告:\n", classification_report(y_test, y_pred))


In [ ]:
# 特征重要性分析
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# 获取特征名称
feature_names = X_sampled.columns  # 假设 x 是一个 DataFrame

# 可视化特征重要性
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.figure(figsize=(10, 6))
plt.title('特征重要性')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), feature_names[indices], rotation=45, ha='right')  # 使用特征名称
plt.xlabel('相对重要性')
plt.show()

In [102]:
print(y_sampled)

In [125]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, roc_auc_score
import matplotlib.pyplot as plt

# 假设 x 是特征数据（DataFrame），y_sampled 是目标变量
# 标准化数值特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 初始化 XGBoost 分类器（使用默认参数）
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# 训练模型
xgb_model.fit(X_train, y_train)

# 在测试集上评估模型
y_pred = xgb_model.predict(X_test)
print("测试集 F1 分数:", f1_score(y_test, y_pred))
print("分类报告:\n", classification_report(y_test, y_pred))

# 特征重要性分析
importances = xgb_model.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns  # 假设 x 是一个 DataFrame

# 可视化特征重要性
plt.figure(figsize=(10, 6))
plt.title('特征重要性')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), feature_names[indices], rotation=45, ha='right')
plt.xlabel('相对重要性')
plt.show()

In [126]:
# 特征重要性分析
importances = xgb_model.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns  # 假设 x 是一个 DataFrame

# 可视化特征重要性
plt.figure(figsize=(10, 6))
plt.title('特征重要性')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), feature_names[indices], rotation=45, ha='right')
plt.xlabel('相对重要性')
plt.show()

In [111]:
df_test=pd.read_csv('dataframe_test.csv')
df_test[selected_features]

In [119]:
X=pre(df_test[selected_features])
y_pred=xgb_model.predict(X)

In [120]:
order_test = [i for i in range(len(df_test))]

# 创建结果 DataFrame
result_df = pd.DataFrame({
    'order': order_test,
    'action_type_DELIVER': y_pred
})

# 按照 order 递增排序
result_df = result_df.sort_values(by='order')

# 保存到 CSV 文件
result_df.to_csv('prediction_results.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,          # 测试集占30%
    random_state=42,        # 固定随机种子
    stratify=y              # 保持类别分布一致
)

# 6. 标准化数值特征（仅需对训练集拟合）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("训练集大小:", X_train.shape[0])
print("测试集大小:", X_test.shape[0])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# 训练逻辑回归模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

# 预测测试集
y_pred = model.predict(X_test_scaled)

# 计算F1分数
f1 = f1_score(y_test, y_pred)  # 假设DELIVERY为1
print("测试集F1分数:", f1)